***
__SingleConvMask & ConvMask__
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geeproduct
import geemask

__SingleConvMask__

In [ ]:
eepoint       = geeutils.bobspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = eedatefrom.advance(1, 'day')
roidiameter   = 64
eeroi         = geeutils.squareareaboundsroi(eepoint, roidiameter/2*20)
verbose       = False
#
# use s2rgb and s2ndvi product as background
#
s2rgbimage    = geeproduct.GEECol_s2rgb().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()
print (geeutils.szprojectioninfo(s2rgbimage))
print (geeutils.szestimatevaluesinfo(s2rgbimage.clip(eeroi))) # clip; otherwise unbounded

s2ndviimage    = geeproduct.GEECol_s2ndvi().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()
print (geeutils.szprojectioninfo(s2ndviimage))
print (geeutils.szestimatevaluesinfo(s2ndviimage.clip(eeroi))) # clip; otherwise unbounded

#
# making scl based masks
#
s2sclimage    = geeproduct.GEECol_s2scl().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()
c1mskimage    = geemask.SingleConvMask([2, 4, 5, 6, 7],   20*9,   -0.057).makemask(s2sclimage)
c2mskimage    = geemask.SingleConvMask([3, 8, 9, 10, 11], 20*101,  0.025).makemask(s2sclimage)

if True:
    #
    # make clear sky transparant, clouds opaque
    #
    # SingleConvMask:
    #    masked:     1 -> updateMask( img.eq(1)) == updateMask(True  (1)) -> img.mask() to 1 : gee clear  (opaque)
    #    clear :     0 -> updateMask( img.eq(1)) == updateMask(False (0)) -> img.mask() to 0 : gee masked (transparent)
    #    no data :   gee masked (default transparent)
    #
    c1mskimage    = c1mskimage.updateMask(c1mskimage.eq(1))
    c2mskimage    = c2mskimage.updateMask(c2mskimage.eq(1))

    
map = geemap.Map(height='600px')
map.centerObject(eepoint, 13)
map.addLayer(s2rgbimage,                                 {'min': 1,  'max':255},                 'rgb')
map.addLayer(s2ndviimage,                                {'min':-1,  'max':1},                  'ndvi')
map.addLayer(s2sclimage,                                 geeutils.s2sclvisParams,                'scl')
map.addLayer(c2mskimage,                                 {'min':0,  'max':1, 'opacity':0.8, 'palette':['#000000','#0000ff']}, 'c2')
map.addLayer(c1mskimage,                                 {'min':0,  'max':1, 'opacity':0.8, 'palette':['#000000','#00ffff']}, 'c1')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 1), {'palette':'#ff0000'})
map

__ConvMask__
- combines multiple SingleConvMask's
- mind the syntax (list-of-list)

In [ ]:
eepoint       = geeutils.bobspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = eedatefrom.advance(1, 'day')
roidiameter   = 64
eeroi         = geeutils.squareareaboundsroi(eepoint, roidiameter/2*20)
verbose       = False
#
# use s2rgb and s2ndvi product as background
#
s2rgbimage    = geeproduct.GEECol_s2rgb().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()
print (geeutils.szprojectioninfo(s2rgbimage))
print (geeutils.szestimatevaluesinfo(s2rgbimage.clip(eeroi))) # clip; otherwise unbounded

s2ndviimage    = geeproduct.GEECol_s2ndvi().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()
print (geeutils.szprojectioninfo(s2ndviimage))
print (geeutils.szestimatevaluesinfo(s2ndviimage.clip(eeroi))) # clip; otherwise unbounded

#
# making scl based masks
#
s2sclimage    = geeproduct.GEECol_s2scl().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()
c1mskimage    = geemask.ConvMask([[2, 4, 5, 6, 7]],   [20*9],   [-0.057]).makemask(s2sclimage)
c2mskimage    = geemask.ConvMask([[3, 8, 9, 10, 11]], [20*101], [0.025]).makemask(s2sclimage)

ccmskimage    = geemask.ConvMask([[2, 4, 5, 6, 7], [3, 8, 9, 10, 11]],
                                  [20*9,            20*101],
                                  [-0.057,           0.025]).makemask(s2sclimage)

if True:
    #
    # make clear sky transparant, clouds opaque
    #
    # ConvMask:
    #    masked:     1 -> updateMask( img.eq(1)) == updateMask(True  (1)) -> img.mask() to 1 : gee clear  (opaque)
    #    clear :     0 -> updateMask( img.eq(1)) == updateMask(False (0)) -> img.mask() to 0 : gee masked (transparent)
    #    no data :   gee masked (default transparent)
    #
    c1mskimage    = c1mskimage.updateMask(c1mskimage.eq(1))
    c2mskimage    = c2mskimage.updateMask(c2mskimage.eq(1))

    ccmskimage    = ccmskimage.updateMask(ccmskimage.eq(1))

    
map = geemap.Map(height='600px')
map.centerObject(eepoint, 13)
map.addLayer(s2rgbimage,                                 {'min': 1,  'max':255},                 'rgb')
map.addLayer(s2ndviimage,                                {'min':-1,  'max':1},                  'ndvi')
map.addLayer(s2sclimage,                                 geeutils.s2sclvisParams,                'scl')
map.addLayer(c2mskimage,                                 {'min':0,  'max':1, 'opacity':0.8, 'palette':['#000000','#0000ff']}, 'c2')
map.addLayer(c1mskimage,                                 {'min':0,  'max':1, 'opacity':0.8, 'palette':['#000000','#00ffff']}, 'c1')
map.addLayer(ccmskimage,                                 {'min':0,  'max':1, 'opacity':0.7, 'palette':['#000000','#ff0000']}, 'cc')
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 1), {'palette':'#ff0000'})
map